In [1]:
import json
import os
import psycopg2
from psycopg2 import OperationalError
from openai import AzureOpenAI
import pandas as pd

from dotenv import load_dotenv
load_dotenv()


True

In [2]:
!pip install pandas

In [3]:
!pip install dotenv

In [4]:
%pip install psycopg2-binary openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
def get_postgres_connection():
    try:
        conn = psycopg2.connect(
            host=os.getenv("POSTGRES_HOST"),
            port=os.getenv("POSTGRES_PORT"),
            dbname=os.getenv("POSTGRES_DB"),
            user=os.getenv("POSTGRES_USER"),
            password=os.getenv("POSTGRES_PASSWORD")
        )
        return conn
    except OperationalError as e:
        print(f"Database connection failed: {e}")
        raise


get_postgres_connection()

<connection object at 0x1167f5540; dsn: 'user=postgres password=xxx dbname=usecase_db host='' port=5432', closed: 0>

In [6]:
def get_matadata():
    metadata_file_path = os.path.join(os.curdir, "metadata", "causal_inference_metadata.txt")
    try:
        with open(metadata_file_path, "r") as f:
            data = f.read()
            print(data)
            return data
    except FileNotFoundError:
        print("File not found")
        return None
    #     print("Invalid JSON format")

In [7]:
def get_llm():
    """
    Initialize and return Azure OpenAI client.
    
    Requires environment variables:
    - AZURE_OPENAI_API_KEY: Your Azure OpenAI API key
    - AZURE_OPENAI_ENDPOINT: Your Azure OpenAI endpoint URL
    - AZURE_OPENAI_DEPLOYMENT_NAME: Your deployment name (model name)
    - AZURE_OPENAI_API_VERSION: API version
    """
    try:
        client = AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
        )
        return client
    except Exception as e:
        print(f"Failed to initialize Azure OpenAI client: {e}")
        print("Please ensure the following environment variables are set:")
        print("- AZURE_OPENAI_API_KEY")
        print("- AZURE_OPENAI_ENDPOINT")
        print("- AZURE_OPENAI_DEPLOYMENT_NAME (optional, can be passed when making calls)")
        raise
    

get_llm()

In [8]:
def generate_sql(input_text):
    metadata = get_matadata()

    sys_prompt = """
    You are an expert SQL query generator.
You will be given:
A natural language question
Database metadata (tables, columns, data types, relationships)
Your task is to generate only a single valid SQL query that answers the question.
Mandatory Rules:
Always fully qualify table names using the causal_inference schema
(e.g., causal_inference.table_name)
Never assume or use the public schema
Use only tables and columns explicitly defined in the metadata
Infer joins strictly from provided relationships
Do not add assumptions or fabricate fields
Do not include explanations, comments, markdown, or extra text
Output only executable SQL
Use ANSI SQL unless otherwise specified
    """
    user_prompt = f"""
    Question: {input_text}
    Database schema metadata: {metadata}
    """

    llm_obj = get_llm()
    response = llm_obj.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )   

    return response.choices[0].message.content

    
    

In [9]:
def validate_sql_query(sql_query):
    """
    Validate a SQL query using LLM and return binary result (True/False).
    
    This function:
    - Removes markdown code blocks (```sql ... ```)
    - Uses LLM to validate SQL against database schema
    - Returns True if valid, False if invalid
    
    Args:
        sql_query: Raw SQL query string (may contain markdown formatting)
    
    Returns:
        tuple: (is_valid: bool, cleaned_query: str)
            - is_valid: True if query is valid, False otherwise
            - cleaned_query: Cleaned SQL query string
    """
    if not sql_query or not isinstance(sql_query, str):
        return False, ""
    
    # Remove markdown code blocks if present
    cleaned_query = sql_query.strip()
    
    # Remove markdown code fences (```sql, ```, etc.)
    if cleaned_query.startswith('```'):
        # Find the first newline after ```
        first_newline = cleaned_query.find('\n')
        if first_newline != -1:
            cleaned_query = cleaned_query[first_newline + 1:]
        else:
            # No newline, just remove the ```
            cleaned_query = cleaned_query[3:]
    
    # Remove closing ```
    if cleaned_query.endswith('```'):
        cleaned_query = cleaned_query[:-3]
    
    # Strip whitespace
    cleaned_query = cleaned_query.strip()
    
    # Basic validation - check if query is empty
    if not cleaned_query:
        return False, cleaned_query
    
    # Get metadata for validation
    metadata = get_matadata()
    llm_obj = get_llm()
    
    sys_prompt = """
    You are an expert SQL validator. Your task is to validate SQL queries against a database schema.
    
    Analyze the provided SQL query and check:
    1. Syntax correctness (valid SQL syntax)
    2. Schema compliance (tables and columns exist in the metadata)
    3. Schema qualification (tables use causal_inference schema)
    4. Column existence (all referenced columns exist in their respective tables)
    5. Join correctness (joins are valid based on relationships)
    6. Data type compatibility (operations are compatible with column types)
    
    Respond with ONLY a single word: "VALID" if the query is valid, or "INVALID" if it has any issues.
    Do not provide explanations, just the single word response.
    """
    
    user_prompt = f"""
    Database schema metadata:
    {metadata}
    
    SQL Query to validate:
    {cleaned_query}
    
    Is this SQL query valid? Respond with only "VALID" or "INVALID".
    """
    
    try:
        response = llm_obj.chat.completions.create(
            model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1  # Low temperature for consistent validation
        )
        
        validation_result = response.choices[0].message.content.strip().upper()
        
        # Check if response indicates valid query
        is_valid = "VALID" in validation_result and "INVALID" not in validation_result
        
        return is_valid, cleaned_query
    
    except Exception as e:
        print(f"Error during LLM validation: {e}")
        # Fallback: return False if LLM validation fails
        return False, cleaned_query

In [10]:
def execute_sql(sql_query):
    """
    Execute a SQL query and return the results.
    
    Args:
        sql_query: SQL query string to execute (will be validated and cleaned)
    
    Returns:
        list: List of dictionaries, where each dictionary represents a row
              with column names as keys
    
    Raises:
        ValueError: If the SQL query is invalid
    """
    try:
        
        conn = get_postgres_connection()
        cursor = conn.cursor()
        cursor.execute(sql_query)
        
        # Get column names
        columns = [desc[0] for desc in cursor.description]
        
        # Fetch all rows and convert to list of dictionaries
        rows = cursor.fetchall()
        results = [dict(zip(columns, row)) for row in rows]
        
        cursor.close()
        conn.close()
        return results
    
    except Exception as e:
        print(f"Error executing SQL query: {e}")
        print(f"Query: {sql_query}")
        raise
    

In [11]:
def generate_summary(data):
    """
    Generate a concise 4-5 line summary of the given data using LLM.
    
    Args:
        data: List of dictionaries representing query results, or pandas DataFrame
    
    Returns:
        str: A 4-5 line summary of the data insights
    """
    if not data:
        return "No data available to summarize."
    
    # Convert to DataFrame if it's a list of dictionaries
    if isinstance(data, list):
        df = pd.DataFrame(data)
    elif isinstance(data, pd.DataFrame):
        df = data.copy()
    else:
        return "Invalid data format. Expected list of dictionaries or DataFrame."
    
    if df.empty:
        return "The query returned no results."
    
    # Get basic statistics about the data
    num_rows = len(df)
    num_cols = len(df.columns)
    column_names = list(df.columns)
    
    # Convert data to a readable format for LLM
    # Limit to first 10 rows to avoid token limits
    sample_data = df.head(10).to_dict('records')
    
    # Get summary statistics for numeric columns
    numeric_summary = {}
    for col in df.select_dtypes(include=['number']).columns:
        numeric_summary[col] = {
            'min': float(df[col].min()),
            'max': float(df[col].max()),
            'mean': float(df[col].mean()),
            'sum': float(df[col].sum())
        }
    
    llm_obj = get_llm()
    
    sys_prompt = """
    You are a data analyst expert. Your task is to analyze query results and generate a concise,
    insightful summary in exactly 4-5 lines.
    
    Rules:
    - Write exactly 4 lines (not more, not less)
    - Focus on key insights, trends, and important findings
    - Use clear, business-friendly language
    - Highlight the most significant data points
    - If there are comparisons (e.g., historical vs current), emphasize the differences
    - Be specific with numbers when relevant
    - Do not include markdown formatting or bullet points
    - Write in paragraph form, with each line being a complete sentence
    """
    
    user_prompt = f"""
    Analyze the following query results and generate a 4-5 line summary:
    
    Data Overview:
    - Number of rows: {num_rows}
    - Number of columns: {num_cols}
    - Column names: {', '.join(column_names)}
    
    Sample data (first {min(10, num_rows)} rows):
    {json.dumps(sample_data, indent=2, default=str)}
    
    Numeric column statistics:
    {json.dumps(numeric_summary, indent=2) if numeric_summary else "No numeric columns"}
    
    Generate a concise 4-5 line summary that highlights the key insights from this data.
    """
    
    try:
        response = llm_obj.chat.completions.create(
            model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3  # Moderate temperature for balanced creativity and consistency
        )
        
        summary = response.choices[0].message.content.strip()
        
        # Ensure it's approximately 4-5 lines (split by periods or newlines)
        lines = [line.strip() for line in summary.split('\n') if line.strip()]
        if len(lines) < 4:
            # If too few lines, try splitting by sentences
            sentences = [s.strip() for s in summary.split('.') if s.strip()]
            if len(sentences) >= 4:
                summary = '. '.join(sentences[:5]) + '.' if len(sentences) > 5 else '. '.join(sentences) + '.'
        elif len(lines) > 5:
            # If too many lines, take first 5
            summary = '\n'.join(lines[:5])
        
        return summary
    
    except Exception as e:
        print(f"Error generating summary: {e}")
        # Fallback to basic summary
        return f"Query returned {num_rows} rows with {num_cols} columns: {', '.join(column_names)}."

In [12]:
def generate_chart():
    pass

In [13]:
def get_suggested_que(count=3):
    """
    Generate suggested questions based on the database metadata.
    
    Args:
        count: Number of suggested questions to generate (default: 3)
    
    Returns:
        list: List of suggested questions as strings
    """
    metadata = get_matadata()
    llm_obj = get_llm()
    
    sys_prompt = """
    You are an expert at analyzing database schemas and generating relevant business questions.
    Based on the provided database metadata, generate natural language questions that users might ask
    to analyze their business data.
    
    Rules:
    - Generate questions that are relevant to the tables, columns, and relationships described in the metadata
    - Questions should be business-focused and actionable
    - Questions should be clear and specific
    - Each question should be on a separate line
    - Do not include numbering, bullets, or markdown formatting
    - Output only the questions, one per line
    """
    
    user_prompt = f"""
    Database schema metadata: {metadata}
    
    Generate exactly {count} relevant business questions that can be answered using this database.
    Output only the questions, one per line, without any numbering or formatting.
    """
    
    try:
        response = llm_obj.chat.completions.create(
            model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        
        # Extract questions from response
        questions_text = response.choices[0].message.content.strip()
        
        # Split by newlines and clean up
        questions = [q.strip() for q in questions_text.split('\n') if q.strip()]
        
        # Remove any numbering or bullets if present
        questions = [q.lstrip('0123456789.-) ').strip() for q in questions]
        
        # Return the requested number of questions
        return questions[:count]
    
    except Exception as e:
        print(f"Error generating suggested questions: {e}")
        raise

In [14]:
os.getenv("AZURE_OPENAI_MODEL_NAME")

'gpt-4o'

In [15]:
recommended_ques = get_suggested_que()
for que in recommended_ques:
    print(que)

"SCHEMA_NAME""causal_inference"

historical_sales:### This table is relevant for questions like: - “What are my top or worst performing categories?” - “What are the top performing products in category(e.g.,'Fruits and Vegetables','Pantry','Meat and Seafood','Bakery and Bread','Frozen Food','Deli and Prepared Food','Dairy and Eggs','Beverages')?” - “What are the worst performing products in category(e.g.,'Fruits and Vegetables','Pantry','Meat and Seafood','Bakery and Bread','Frozen Food','Deli and Prepared Food','Dairy and Eggs','Beverages')?” - “How did my categories perform last week?” ### Table description: The `historical_sales` table contains **previous year’s sales data** across items, stores, and categories. It is **essential for comparing current sales to last year’s**, helping to measure trends, growth, or decline over time. ### When to Use: Use this table for any analysis that requires **year-over-year (YoY) comparison**, especially to: - Compare last week’s sales vs. the same

In [21]:
input_text = "How do historical sales compare to current year sales for the Beverages category across all stores?"

conn = get_postgres_connection()
generated_sql = generate_sql(input_text)
print(generated_sql)

"SCHEMA_NAME""causal_inference"

historical_sales:### This table is relevant for questions like: - “What are my top or worst performing categories?” - “What are the top performing products in category(e.g.,'Fruits and Vegetables','Pantry','Meat and Seafood','Bakery and Bread','Frozen Food','Deli and Prepared Food','Dairy and Eggs','Beverages')?” - “What are the worst performing products in category(e.g.,'Fruits and Vegetables','Pantry','Meat and Seafood','Bakery and Bread','Frozen Food','Deli and Prepared Food','Dairy and Eggs','Beverages')?” - “How did my categories perform last week?” ### Table description: The `historical_sales` table contains **previous year’s sales data** across items, stores, and categories. It is **essential for comparing current sales to last year’s**, helping to measure trends, growth, or decline over time. ### When to Use: Use this table for any analysis that requires **year-over-year (YoY) comparison**, especially to: - Compare last week’s sales vs. the same

In [22]:
# Validate and clean the SQL query using LLM
is_valid, cleaned_query = validate_sql_query(generated_sql)
print(is_valid, cleaned_query)

if not is_valid:
    raise ValueError("SQL query validation failed. The query is invalid according to the database schema.")

if not cleaned_query:
    raise ValueError("SQL query is empty after cleaning")

"SCHEMA_NAME""causal_inference"

historical_sales:### This table is relevant for questions like: - “What are my top or worst performing categories?” - “What are the top performing products in category(e.g.,'Fruits and Vegetables','Pantry','Meat and Seafood','Bakery and Bread','Frozen Food','Deli and Prepared Food','Dairy and Eggs','Beverages')?” - “What are the worst performing products in category(e.g.,'Fruits and Vegetables','Pantry','Meat and Seafood','Bakery and Bread','Frozen Food','Deli and Prepared Food','Dairy and Eggs','Beverages')?” - “How did my categories perform last week?” ### Table description: The `historical_sales` table contains **previous year’s sales data** across items, stores, and categories. It is **essential for comparing current sales to last year’s**, helping to measure trends, growth, or decline over time. ### When to Use: Use this table for any analysis that requires **year-over-year (YoY) comparison**, especially to: - Compare last week’s sales vs. the same

In [23]:
data = execute_sql(cleaned_query)
df = pd.DataFrame(data)
df

,category_name,historical_sales_amount,current_sales_amount
0,Beverages,1271079471345.60,9.864692e+11


In [24]:
summary = generate_summary(data)
summary


'The data focuses on the "Beverages" category, showing a significant decline in sales from a historical amount of $1. 27 trillion to a current amount of $986. 47 billion. This represents a decrease of approximately 22. 4%.'

In [ ]:
# chart = generate_chart(data)